In [6]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

data = pd.read_csv('data/train.csv')
data

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00,42.9,0.718,15.7,73.1,64.4
1,2020-01-01 01,33.6,0.587,10.1,74.8,66.0
2,2020-01-01 02,29.3,NaN,5.1,51.0,44.9
3,2020-01-01 03,30.5,0.246,7.2,27.7,25.1
4,2020-01-01 04,29.3,0.204,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18,NaN,0.222,55.1,12.0,5.3
40987,2024-09-03 19,NaN,0.245,48.2,13.4,7.0
40988,2024-09-03 20,NaN,0.234,44.5,12.4,7.1
40989,2024-09-03 21,NaN,0.225,25.9,10.6,5.4


In [13]:
data_long = pd.melt(data, id_vars=['id'], value_vars=['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
                    var_name='Pollutant', value_name='Value')
train_data = TimeSeriesDataFrame.from_data_frame(
    data_long,
    id_column="Pollutant",
    timestamp_column="id"
)
train_data.head()

Value
item_id    timestamp                 
valeur_NO2 2020-01-01 00:00:00   42.9
           2020-01-01 01:00:00   33.6
           2020-01-01 02:00:00   29.3
           2020-01-01 03:00:00   30.5
           2020-01-01 04:00:00   29.3

In [15]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="Value",
    eval_metric="MAE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          8
GPU Count:          0
Memory Avail:       11.29 GB / 31.70 GB (35.6%)
Disk Space Avail:   68.51 GB / 476.00 GB (14.4%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'Value',
 'time_limit': 600,
 'verbosity': 2}

Inferred time series frequency: 'H'
train_data contains missing values represented by NaN. They have been filled by carrying forward the last valid obser

In [20]:
test_data = pd.read_csv('data/test.csv')
test_data["id"] = pd.to_datetime(test_data["id"])
pollutants = ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
test_data = test_data.loc[test_data.index.repeat(len(pollutants))]
test_data['Pollutant'] = pollutants * (len(test_data) // len(pollutants))
test_data

,id,Pollutant
0,2024-09-03 23:00:00,valeur_NO2
0,2024-09-03 23:00:00,valeur_CO
0,2024-09-03 23:00:00,valeur_O3
0,2024-09-03 23:00:00,valeur_PM10
0,2024-09-03 23:00:00,valeur_PM25
...,...,...
503,2024-09-24 22:00:00,valeur_NO2
503,2024-09-24 22:00:00,valeur_CO
503,2024-09-24 22:00:00,valeur_O3
503,2024-09-24 22:00:00,valeur_PM10


In [19]:
pd.read_csv('data/test.csv')

,id
0,2024-09-03 23
1,2024-09-04 00
2,2024-09-04 01
3,2024-09-04 02
4,2024-09-04 03
...,...
499,2024-09-24 18
500,2024-09-24 19
501,2024-09-24 20
502,2024-09-24 21
